In [2]:
import math
import jax
import jax.numpy as np
from jax import grad, jit, vmap, jacfwd
import dhutils.dhutils as dhu
import pythreejs as THREE

In [3]:
# Implementation based on ideas from
# https://cg.informatik.uni-freiburg.de/course_notes/sim_03_cloth1.pdf

# define number of edges along each direction
# assumption: square cloth
N = 4

# generate the mesh and get (u, v), (x, y, z) coordinates
Nx = N
Ny = N

In [4]:
# define the initial conditions: x0, v0
# define x0
cloth_length = 1.5
x0, faces = dhu.standard_rectangle(cloth_length, cloth_length, Nx, Ny)

# define b_u, b_v
b_u = cloth_length / Nx
b_v = cloth_length / Ny

# define v0
v0_0 = np.zeros(((N+1)**2, 3))
#v0_1 = v0_0.at[0].set([-0.2, -0.1, 0.3])
#v0_2 = v0_1.at[int(((N+1)**2-1)/2)].set([0, 0, 1.5])
#v0 = v0_2.at[-1].set([0.5, 0.4, -0.1])
v0 = v0_0

# inspect faces
print("faces:")
print(faces)

# inspect x0
print("x0:")
print(x0)

# inspect v0
print("v0:")
print(v0)

faces:
[[ 0  1  6]
 [ 0  6  5]
 [ 1  2  7]
 [ 1  7  6]
 [ 2  3  8]
 [ 2  8  7]
 [ 3  4  9]
 [ 3  9  8]
 [ 5  6 11]
 [ 5 11 10]
 [ 6  7 12]
 [ 6 12 11]
 [ 7  8 13]
 [ 7 13 12]
 [ 8  9 14]
 [ 8 14 13]
 [10 11 16]
 [10 16 15]
 [11 12 17]
 [11 17 16]
 [12 13 18]
 [12 18 17]
 [13 14 19]
 [13 19 18]
 [15 16 21]
 [15 21 20]
 [16 17 22]
 [16 22 21]
 [17 18 23]
 [17 23 22]
 [18 19 24]
 [18 24 23]]
x0:
[[ 0.    -0.     0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.    -0.75   0.   ]
 [ 0.    -1.125  0.   ]
 [ 0.    -1.5    0.   ]
 [ 0.375 -0.     0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.375 -0.75   0.   ]
 [ 0.375 -1.125  0.   ]
 [ 0.375 -1.5    0.   ]
 [ 0.75  -0.     0.   ]
 [ 0.75  -0.375  0.   ]
 [ 0.75  -0.75   0.   ]
 [ 0.75  -1.125  0.   ]
 [ 0.75  -1.5    0.   ]
 [ 1.125 -0.     0.   ]
 [ 1.125 -0.375  0.   ]
 [ 1.125 -0.75   0.   ]
 [ 1.125 -1.125  0.   ]
 [ 1.125 -1.5    0.   ]
 [ 1.5   -0.     0.   ]
 [ 1.5   -0.375  0.   ]
 [ 1.5   -0.75   0.   ]
 [ 1.5   -1.125  0.   ]
 [ 1.5   -1.5    0.   ]]
v0:

In [5]:
# define the mass matrix
# we assume all particles have the same mass m
# 3*(N+1)^2, 3*(N+1)^2
m_per_particle = 0.2
M = np.diag(m_per_particle*np.ones((3*(N+1)**2)))

# check M
print(M)

[[0.2 0.  0.  ... 0.  0.  0. ]
 [0.  0.2 0.  ... 0.  0.  0. ]
 [0.  0.  0.2 ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.2 0.  0. ]
 [0.  0.  0.  ... 0.  0.2 0. ]
 [0.  0.  0.  ... 0.  0.  0.2]]


In [6]:
# define k_stretch, k_shear and k_bend
k_stretch = 0.5
k_shear = 0.5
k_bend = 0.5

In [7]:
# for each triangle, construct u, v matrix
# N+1, N+1
#uu = range(0, N+1)
#vv = range(0, N+1)
#u, v = torch.meshgrid(uu, vv)
#print(u)
#print(v)

# define helper functions
def get_u(vertex_index, Ny):
    r"""
    Compute u from vertex index.
    """
    return float(np.floor(vertex_index / (Ny+1)))

def get_v(vertex_index, Nx):
    r"""
    Compute v from vertex index.
    """
    return float(np.floor(vertex_index % (Nx+1)))

def get_face_indices(vertex_index, faces):
    r"""
    Get a list of face indices that a vertex belongs
    to.
    """
    face_index_list = []
    for face_index in range(faces.shape[0]):
        face = faces[face_index]
        if vertex_index in face:
            face_index_list.append(face_index)
    return face_index_list

# check
print(get_u(4, 2))
print(get_face_indices(2, np.array([[0, 1, 3], [2, 9, 10], [1, 2, 5]])))

1.0
[1, 2]


In [8]:
# define del_u, del_v for each triangle
# 2*N^2, 2
# they should remain constant regardless
# of changes to world positions
assert 2*N**2 == faces.shape[0]
del_u_0 = np.zeros((2*N**2, 2))
del_v_0 = np.zeros((2*N**2, 2))
del_us = [del_u_0]
del_vs = [del_v_0]

for face_index in range(faces.shape[0]):
    triangle = faces[face_index]
    # define [u, v] for point i, j, k
    # we define the first point to be i, 2nd to be j,
    # 3rd to be k
    uv_i = np.array([get_u(triangle[0], N), get_v(triangle[0], N)])
    uv_j = np.array([get_u(triangle[1], N), get_v(triangle[1], N)])
    uv_k = np.array([get_u(triangle[2], N), get_v(triangle[2], N)])
    # compute (2, ) del_u
    del_us.append(del_us[face_index].at[face_index].set([uv_j[0]-uv_i[0], uv_k[0]-uv_i[0]]))
    # compute (2, ) del_v
    del_vs.append(del_vs[face_index].at[face_index].set([uv_j[1]-uv_i[1], uv_k[1]-uv_i[1]]))

del_u = del_us[-1]
del_v = del_vs[-1]

# compute area of triangles
# 2*N^2
area_0 = np.zeros((2*N**2))
areas = [area_0]
for face_index in range(faces.shape[0]):
    areas.append(areas[face_index].at[face_index].set(
        abs(0.5 * (del_u[face_index, 0]*del_v[face_index, 1] - del_v[face_index, 0]*del_u[face_index, 1]))))

area = areas[-1]

# check del_u, del_v, area
print(del_u)
print(del_v)
print(area)

[[0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]
 [0. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


In [9]:
# define del_x1
def del_x1(x):
    r"""
    Compute del(x1) from x.
    
    :param x: ((N+1)^2, 3) array
    
    Return:
    dx1: (2*N^2, 3) array
    """
    # for each triangle, compute del(x1)
    # 2*N^2, 3
    dx1_0 = np.zeros((2*N**2, 3))
    dx1s = [dx1_0]
    
    for face_index in range(faces.shape[0]):
        # build del_x1: (3, )
        dx1s.append(dx1s[face_index].at[face_index].set(
            x[faces[face_index][1]] - x[faces[face_index][0]]))
        
    dx1 = dx1s[-1]
    return dx1

# define del_x2
def del_x2(x):
    r"""
    Compute del(x2) from x.
    
    :param x: ((N+1)^2, 3) array
    
    Return:
    dx2: (2*N^2, 3) array
    """
    # for each triangle, compute del(x1)
    # 2*N^2, 3
    dx2_0 = np.zeros((2*N**2, 3))
    dx2s = [dx2_0]
    
    for face_index in range(faces.shape[0]):
        # build del_x1: (3, )
        dx2s.append(dx2s[face_index].at[face_index].set(
            x[faces[face_index][2]] - x[faces[face_index][0]]))
        
    dx2 = dx2s[-1]
    return dx2

# define w_u
def dw_du(dx1, dx2):
    r"""
    Compute w_u from dx1 and dx2.
    
    :param dx1: del(x1), (2*N^2, 3) array
    :param dx2: del(x2), (2*N^2, 3) array
    
    Return:
    w_u: (2*N^2, 3) array
    """
    # for each triangle, compute w_u
    # 2*N^2, 3
    w_u_0 = np.zeros((2*N**2, 3))
    w_us = [w_u_0]
    
    for face_index in range(faces.shape[0]):
        # build (2, 2) matrix del_uv = [del_u1 del_u2; del_v1 del_v2]
        del_uv = np.array([del_u[face_index], del_v[face_index]])
        # build (3, 2) matrix del_x = [dx1 dx2]
        dx = np.transpose(np.array([dx1[face_index], dx2[face_index]]))
        # compute (3, ) per-triangle [wu]
        deform = np.matmul(dx, np.linalg.inv(del_uv))
        w_us.append(w_us[face_index].at[face_index].set(deform[:, 0]))
    
    w_u = w_us[-1]
    return w_u

# define w_v
def dw_dv(dx1, dx2):
    r"""
    Compute w_v from dx1 and dx2.
    
    :param dx1: del(x1), (2*N^2, 3) array
    :param dx2: del(x2), (2*N^2, 3) array
    
    Return:
    w_v: (2*N^2, 3) array
    """
    # for each triangle, compute w_v
    # 2*N^2, 3
    w_v_0 = np.zeros((2*N**2, 3))
    w_vs = [w_v_0]
    
    for face_index in range(faces.shape[0]):
        # build (2, 2) matrix del_uv = [del_u1 del_u2; del_v1 del_v2]
        del_uv = np.array([del_u[face_index], del_v[face_index]])
        # build (3, 2) matrix dx = [dx1 dx2]
        dx = np.transpose(np.array([dx1[face_index], dx2[face_index]]))
        # compute (3, ) per-triangle [wu]
        deform = np.matmul(dx, np.linalg.inv(del_uv))
        w_vs.append(w_vs[face_index].at[face_index].set(deform[:, 1]))
    
    w_v = w_vs[-1]
    return w_v

# check del_x1, del_x2
dx1 = del_x1(x0)
dx2 = del_x2(x0)
print(dx1)
print(dx2)

# check dw_du, dw_dv
print(dw_du(dx1, dx2))
print(dw_dv(dx1, dx2))

[[ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.    -0.375  0.   ]
 [ 0.375 -0.375  0.   ]]
[[ 0.375 -0.375  0.   ]
 [ 0.375  0.     0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.375  0.     0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.375  0.     0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.375  0.     0.   ]
 [ 0.375 -0.375  0.   ]
 [ 0.375  0.   

In [10]:
# define stretch energy
def e_stretch(x):
    r"""
    Compute stretch energy for each triangle
    from world position x.
    
    :param x: (3*(N+1)^2, ) array
    
    Return:
    energy matrix of shape (2*N^2, )
    """
    # reshape x to ((N+1)^2, 3)
    x_reshaped = np.reshape(x, ((N+1)**2, 3))
    
    # compute del(x1), del(x2)
    # 2*N^2, 3
    dx1 = del_x1(x_reshaped)
    dx2 = del_x2(x_reshaped)
    
    # compute w_u, w_v
    # 2*N^2, 3
    w_u = dw_du(dx1, dx2)
    w_v = dw_dv(dx1, dx2)

    # for each triangle, compute condition matrix C_st
    # 2*N^2, 2
    C_st_0 = np.zeros((2*N**2, 2))
    C_sts = [C_st_0]
    for face_index in range(faces.shape[0]):
        C_sts.append(C_sts[face_index].at[face_index].set(
            area[face_index] * np.array([np.linalg.norm(w_u[face_index])-b_u, np.linalg.norm(w_v[face_index])-b_v])))
    
    C_st = C_sts[-1]

    # for each triangle, compute energy matrix E_st
    # 2*N^2
    E_st_0 = np.zeros((2*N**2))
    E_sts = [E_st_0]
    for face_index in range(faces.shape[0]):
        E_sts.append(E_sts[face_index].at[face_index].set(
            0.5 * k_stretch * np.dot(C_st[face_index], C_st[face_index])))
    E_st = E_sts[-1]
        
    return E_st

# define f_stretch
def f_stretch(x):
    r"""
    Compute stretch force on particles x
    due to all triangles (that each particle is in).
    
    :param x: ((N+1)^2, 3) array
    
    Return:
    Stretch force ((N+1)^2, 3)
    """
    # flatten x to (3*(N+1)^2, )
    x_flat = np.reshape(x, (3*(N+1)**2,))
    
    # compute Jacobian dE/dx
    # 2*N^2, 3*(N+1)^2
    dEdx = jacfwd(e_stretch)(x_flat)
    
    # sum up gradients over all triangles
    # 3*(N+1)^2, 
    f_st = np.sum(dEdx, axis=0)
    
    # reshape to (N+1)^2, 3
    f_st_reshaped = np.reshape(f_st, ((N+1)**2, 3))
    return f_st_reshaped

# check e_stretch
e_st = e_stretch(x0)
print(e_st)

# check f_stretch
f_st = f_stretch(x0)
print(f_st)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [13]:
# define shear energy
def e_shear(x):
    r"""
    Compute shear energy for each triangle
    from world position x.
    
    :param x: (3*(N+1)^2, ) array
    
    Return:
    energy matrix of shape (2*N^2, )
    """
    # reshape x to ((N+1)^2, 3)
    x_reshaped = np.reshape(x, ((N+1)**2, 3))
    
    # compute del(x1), del(x2)
    # 2*N^2, 3
    dx1 = del_x1(x_reshaped)
    dx2 = del_x2(x_reshaped)
    
    # compute w_u, w_v
    # 2*N^2, 3
    w_u = dw_du(dx1, dx2)
    w_v = dw_dv(dx1, dx2)
    
    # compute C_sh = area * w_u^T * w_v
    # 2*N^2, 
    C_sh = np.multiply(area, np.sum(np.multiply(w_u, w_v), axis=1))
    
    # compute e_sh = 0.5 * k_shear * C_sh^T * C_sh
    e_sh = 0.5 * k_shear * np.multiply(C_sh, C_sh)
    return e_sh

# define f_shear
def f_shear(x):
    r"""
    Compute shear force on particles x
    due to all triangles (that each particle is in).
    
    :param x: ((N+1)^2, 3) array
    
    Return:
    Shear force ((N+1)^2, 3)
    """
    # flatten x to (3*(N+1)^2, )
    x_flat = np.reshape(x, (3*(N+1)**2,))
    
    # compute Jacobian dE/dx
    # 2*N^2, 3*(N+1)^2
    dEdx = jacfwd(e_shear)(x_flat)
    
    # sum up gradients over all triangles
    # 3*(N+1)^2, 
    f_sh = np.sum(dEdx, axis=0)
    
    # reshape to (N+1)^2, 3
    f_sh_reshaped = np.reshape(f_sh, ((N+1)**2, 3))
    return f_sh_reshaped

# check f_shear
x1 = x0.copy()
#x1[0] = [-0.1, -0.1, 0]
x1[12] = [0, 0, 1.0]
f_sh = f_shear(x1)
print(f_sh)

[[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.03955078  0.03955078  0.0703125 ]
 [ 0.21240234 -0.3383789  -0.30078125]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.3383789  -0.21240234 -0.30078125]
 [-0.8232422   0.8232422   1.0625    ]
 [ 0.08642578 -0.15966797 -0.23046875]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.15966797 -0.08642578 -0.23046875]
 [ 0.06591797 -0.06591797 -0.0703125 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


In [15]:
# define function to compute face normal
def get_normals(x):
    r"""
    Compute normals at the faces.
    
    :param x: world positions of particles ((N+1)^2, 3)
    
    Return:
    face normals of shape (2*N^2, 3)
    """
    # get dx1 and dx2
    # 2*N^2, 3
    dx1 = del_x1(x)
    dx2 = del_x2(x)
    
    # get the cross product dx1 x dx2
    # 2*N^2, 3
    normals = np.cross(dx1, dx2)
    
    # get the normalized normals
    # 2*N^2, 3
    norms = np.linalg.norm(normals, axis=1)
    normals_normalized_0 = np.zeros(normals.shape)
    normals_normalized_arr = [normals_normalized_0]
    for face_index in range(faces.shape[0]):
        normals_normalized_arr.append(normals_normalized_arr[face_index].at[face_index].set(
            normals.at[face_index].get()/norms.at[face_index].get()
        ))
    normals_normalized = normals_normalized_arr[-1]
    return normals_normalized        

# define function to produce pairs of adjacent faces
def get_pairs_of_adj_faces():
    r"""
    Get pairs of faces that share an edge.
    
    Return:
    array of face pairs with shape (3*N^2-2*N, 2)
    """
    # we take advantage of the fact that
    # we have a square cloth with preset
    # geometry
    assert faces.shape[0] == 2*N**2
    face_pairs_0 = np.zeros((3*N**2-2*N, 2))
    face_pairs_arr = [face_pairs_0]
    
    # first we identify face pairs sharing
    # diagonal edges    
    for pair_index in range(N**2):
        face_index = pair_index
        face_pairs_arr.append(face_pairs_arr[pair_index].at[pair_index].set(
            [2*face_index, 2*face_index+1]
        ))
    
    # then we identify face pairs sharing
    # horizontal edges
    column_index = 0
    for pair_index in range(N**2, N**2+N*(N-1), N-1):
        # identify the index of the first face along a column
        face_index = column_index * 2 * N
        # add pairs along the column
        for i in range(N-1):
            pair_index_offseted = pair_index + i
            face_pairs_arr.append(face_pairs_arr[pair_index_offseted].at[pair_index_offseted].set(
                [face_index+2*i, face_index+2*i+3]
            ))
        column_index += 1
    
    # at last we identify face pairs sharing
    # vertical edges
    row_index = 0
    for pair_index in range(N**2+N*(N-1), 3*N**2-2*N, N-1):
        # identify the index of the first face along a row
        face_index = row_index * 2 + 1
        # add pairs along the row
        for i in range(N-1):
            pair_index_offseted = pair_index + i
            face_pairs_arr.append(face_pairs_arr[pair_index_offseted].at[pair_index_offseted].set(
                [face_index+(2*N)*i, face_index+(2*N)*i+(2*N-1)]
            ))
        row_index += 1
    
    face_pairs = face_pairs_arr[-1]
    return face_pairs

# find pairs of adjacent triangles
# 2*N^2, 2
adj_faces = get_pairs_of_adj_faces()

# define bend energy
def e_bend(x):
    r"""
    Compute bend energy on each pair of
    adjacent triangles due to world position
    x.
    
    :param x: (3*(N+1)^2, ) array
    
    Return:
    energy matrix of shape (3*N^2-2N, )
    """
    # reshape x to ((N+1)^2, 3)
    x_reshaped = np.reshape(x, ((N+1)**2, 3))
    
    # for each pair, compute energy
    face_normals = get_normals(x_reshaped)
    E_be_0 = np.zeros((3*N**2-2*N, ))
    E_bes = [E_be_0]
    
    for pair_index in range(len(adj_faces)):
        # compute C_be for the pair
        face_pair = adj_faces[pair_index]
        n1 = face_normals[int(face_pair[0])]
        n2 = face_normals[int(face_pair[1])]
        C_be_pair = np.arccos(np.dot(n1, n2))
        # compute E_be for the pair
        E_bes.append(E_bes[pair_index].at[pair_index].set(0.5 * k_bend * C_be_pair**2))
    E_be = E_bes[-1]
    return E_be
        
# define f_bend
def f_bend(x):
    r"""
    Compute bend force on particles x
    due to all triangle pairs (for which the
    shared edge is connected to the particle.)
    
    :param x: ((N+1)^2, 3) array
    
    Return:
    Bend force ((N+1)^2, 3)
    """
    # flatten x to (3*(N+1)^2, )
    x_flat = np.reshape(x, (3*(N+1)**2,))
    
    # compute Jacobian dE/dx
    # 3*N^2-2*N, 3*(N+1)^2
    dEdx = jacfwd(e_bend)(x_flat)
    
    # sum up gradients over all triangles
    # NOTE: we only consider non-nan gradients
    # 3*(N+1)^2, 
    f_be = np.nansum(dEdx, axis=0)
    
    # reshape to (N+1)^2, 3
    f_be_reshaped = np.reshape(f_be, ((N+1)**2, 3))
    return f_be_reshaped

# check get_normals()
#print(get_normals(x0))

# check get_pairs_of_adj_faces()
#get_pairs_of_adj_faces()

# check e_bend(x)
x2 = x0.copy()
#x2[0, 2] = 1.1
#x2[1, 2] = -0.4
#x2[5, 2] = -0.1
#x2[6, 2] = -0.3
#e_be = e_bend(x2)
#print(e_be)
x2[0] = [0, 0, 1.0]

# check f_bend(x)
f_be = f_bend(x2)
print(f_be.shape)
print(f_be)

(25, 3)
[[ 7.4146771e-01 -7.4146771e-01  5.5610073e-01]
 [-5.3837724e-08  2.4461703e+00 -9.1731393e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-2.4461703e+00  5.3837724e-08 -9.1731393e-01]
 [ 1.7047026e+00 -1.7047026e+00 -1.9535418e+00]
 [ 0.0000000e+00  0.0000000e+00  1.6160344e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  1.6160344e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00 

In [18]:
# define external force
def f_external(t):
    r"""
    Return an external force on each particle dependent
    of time.
    
    :param t: time
    
    Return:
    External force ((N+1)^2, 3)
    """
    f_ext_0 = np.zeros(((N+1)**2, 3))
    f_exts = [f_ext_0]
    for vertex_index in range(((N+1)**2)):
        if vertex_index == 12:
            # we apply a constant force at the center of the cloth
            f_exts.append(f_exts[vertex_index].at[vertex_index].set([0, 0, -2.0]))
        else:
            f_exts.append(f_exts[vertex_index].at[vertex_index].set([0, 0, 0]))
    f_ext = f_exts[-1]
    return f_ext

# define f_total (total force)
def f_total(x, t):
    r"""
    Compute total force on particles at position
    x due to stretch, shear and bend.
    
    :param x: (3*(N+1)^2, )
    :param t: time
    
    Return:
    Force array (3*(N+1)^2, )
    """
    # reshape x to ((N+1)^2, 3)
    x_reshaped = np.reshape(x, ((N+1)**2, 3))
    
    # compute stretch
    # (N+1)^2, 3
    f_st = f_stretch(x_reshaped)
    
    # compute shear
    # (N+1)^2, 3
    f_sh = f_shear(x_reshaped)
    
    # compute bend
    f_be = f_bend(x_reshaped)
    
    # compute an external force
    # (N+1)^2, 3
    f_ext = f_external(t)
    
    # add up all forces
    f_tot = f_st + f_sh + f_be + f_ext
    #f_tot = np.zeros(((N+1)**2, 3))
    
    # flatten total force
    f_tot_flat = np.reshape(f_tot, (3*(N+1)**2, ))
    
    return f_tot_flat

# check f_total
f_tot = f_total(x0.reshape((3*(N+1)**2, )), 1.0)
print(f_tot)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. -2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]


In [19]:
# define df/dx
# 3*(N+1)^2, 3*(N+1)^2
def df_dx(x, t):
    r"""
    Compute the total force Jacobian w.r.t. world
    position x.
    
    :param x: ((N+1)^2, 3)
    :param t: time
    
    Return:
    Jacobian df/dx of shape (3*(N+1)^2, 3*(N+1)^2)
    """
    # flatten x
    x_flat = np.reshape(x, (3*(N+1)**2, ))
    
    # compute df/dx
    dfdx = jacfwd(f_total)(x_flat, t)
    return dfdx

# check df_dx
dfdx = df_dx(x0, 1.0)
print(dfdx)

[[0.14257811 0.         0.         ... 0.         0.         0.        ]
 [0.         0.14257811 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.14257811 0.         0.        ]
 [0.         0.         0.         ... 0.         0.14257811 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [20]:
# define df/dv
# since we do not simulate damping,
# we can assume df_dv = 0
df_dv = 0

In [21]:
# define v and x update
def compute_dv(v_curr, x_curr, dt, t):
    r"""
    Compute the change in velocity dv given the
    current velocity, position and time step.
    
    :param v_curr: current velocity, ((N+1)^2, 3)
    :param x_curr: current position, ((N+1)^2, 3)
    :param dt: time step size, scalar
    :param t: time
    
    Return:
    dv, change in velocity, ((N+1)^2, 3)
    """
    # flatten v_curr
    v_curr_flat = np.reshape(v_curr, (3*(N+1)**2, ))
    
    # flatten x_curr
    x_curr_flat = np.reshape(x_curr, (3*(N+1)**2, ))
    
    # we compute df_dx and save it
    dfdx = df_dx(x_curr, t)
    
    # we formulate the problem as A*dv = b 
    # compute A = I - h*inv(M)*df/dv - h^2*inv(M)*df/dx
    A = np.eye(3*(N+1)**2) - dt * df_dv - dt**2 * np.matmul(np.linalg.inv(M), dfdx)
    
    # compute b = h*inv(M)*(f+h*df/dx*v)
    b = dt * np.matmul(np.linalg.inv(M), (f_total(x_curr_flat, t) + dt * np.matmul(dfdx, v_curr_flat)))
    
    # compute dv
    # (N+1)^2, 3
    dv_flat = np.matmul(np.linalg.inv(A), b)
    dv = np.reshape(dv_flat, ((N+1)**2, 3))
    return dv

def compute_v_next(v_curr, dv):
    r"""
    Compute the next velocity given the current
    velocity and the velocity increment.
    
    :param v_curr: current velocity, ((N+1)^2, 3)
    :param dv: velocity change, ((N+1)^2, 3)
    
    Return:
    next velocity, ((N+1)^2, 3)
    """
    return (v_curr + dv)

def compute_x_next(x_curr, v_next, dt):
    r"""
    Compute the new position given the current
    position, the next velocity and dt.
    
    :param x_curr: current position, ((N+1)^2, 3)
    :param v_next: next velocity, ((N+1)^2, 3)
    
    Return:
    next position, ((N+1)^2, 3)
    """
    return (x_curr + dt*v_next)

# check time stepping
dt = 1e-2
t = dt
dv = compute_dv(v0, x0, dt, t)
v1 = compute_v_next(v0, dv)
x1 = compute_x_next(x0, v1, dt)
print(x1)

[[ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00 -3.750000e-01  0.000000e+00]
 [ 0.000000e+00 -7.500000e-01  0.000000e+00]
 [ 0.000000e+00 -1.125000e+00  0.000000e+00]
 [ 0.000000e+00 -1.500000e+00  0.000000e+00]
 [ 3.750000e-01  0.000000e+00  0.000000e+00]
 [ 3.750000e-01 -3.750000e-01  0.000000e+00]
 [ 3.750000e-01 -7.500000e-01  0.000000e+00]
 [ 3.750000e-01 -1.125000e+00  0.000000e+00]
 [ 3.750000e-01 -1.500000e+00  0.000000e+00]
 [ 7.500000e-01  0.000000e+00  0.000000e+00]
 [ 7.500000e-01 -3.750000e-01  0.000000e+00]
 [ 7.500000e-01 -7.500000e-01 -9.999999e-04]
 [ 7.500000e-01 -1.125000e+00  0.000000e+00]
 [ 7.500000e-01 -1.500000e+00  0.000000e+00]
 [ 1.125000e+00  0.000000e+00  0.000000e+00]
 [ 1.125000e+00 -3.750000e-01  0.000000e+00]
 [ 1.125000e+00 -7.500000e-01  0.000000e+00]
 [ 1.125000e+00 -1.125000e+00  0.000000e+00]
 [ 1.125000e+00 -1.500000e+00  0.000000e+00]
 [ 1.500000e+00  0.000000e+00  0.000000e+00]
 [ 1.500000e+00 -3.750000e-01  0.000000e+00]
 [ 1.50000

In [23]:
# define total number of steps, time step size
Nt = 20
dt = 1e-2
times = []

# define positions and velocities
vt = [v0]
xt = [x0]

for k in range(0, Nt, 1):
    x = x0.copy()
    t = k * dt
    # update velocity and world position,
    # given the relation
    # v_(k+1) = v_k + dv
    # x_(k+1) = x_k = dt*v_(k+1)
    dv = compute_dv(vt[k], xt[k], dt, t)
    vt.append(compute_v_next(vt[k], dv))
    x_next = compute_x_next(xt[k], vt[k+1], dt)
    # apply positional constraints
    x_next_constrained_1 = x_next.at[0].set(x0[0])
    x_next_constrained_4 = x_next_constrained_1.at[4].set(x0[4])
    x_next_constrained_20 = x_next_constrained_4.at[20].set(x0[20])
    x_next_constrained_24 = x_next_constrained_20.at[24].set(x0[24])
    xt.append(x_next_constrained_24)
    times.append(t)

dhu.mesh_animation(times, xt, faces)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(5.0, 3.0, 5.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.morphTargetInfluences', times=array([0. …